In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    model  <- train(Species ~ ., data = iris, method = "naive_bayes")
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
pred <- predict(model, as.matrix(iris[, -5]) , type="prob")
pred_labels <- predict(model, as.matrix(iris[, -5]) , type="raw")
sum(pred_labels != iris$Species)/length(pred_labels)


[1] 0.04

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_naive_bayes_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)

WITH centered_data AS 
(SELECT "ADS"."KEY" AS "KEY", CAST("ADS"."Feature_0" AS FLOAT) - 5.006 AS "Feature_0_setosa", CAST("ADS"."Feature_1" AS FLOAT) - 3.428 AS "Feature_1_setosa", CAST("ADS"."Feature_2" AS FLOAT) - 1.462 AS "Feature_2_setosa", CAST("ADS"."Feature_3" AS FLOAT) - 0.24600000000000002 AS "Feature_3_setosa", CAST("ADS"."Feature_0" AS FLOAT) - 5.936 AS "Feature_0_versicolor", CAST("ADS"."Feature_1" AS FLOAT) - 2.77 AS "Feature_1_versicolor", CAST("ADS"."Feature_2" AS FLOAT) - 4.26 AS "Feature_2_versicolor", CAST("ADS"."Feature_3" AS FLOAT) - 1.3259999999999998 AS "Feature_3_versicolor", CAST("ADS"."Feature_0" AS FLOAT) - 6.587999999999999 AS "Feature_0_virginica", CAST("ADS"."Feature_1" AS FLOAT) - 2.9739999999999998 AS "Feature_1_virginica", CAST("ADS"."Feature_2" AS FLOAT) - 5.5520000000000005 AS "Feature_2_virginica", CAST("ADS"."Feature_3" AS FLOAT) - 2.026 AS "Feature_3_virginica" 
FROM "INPUT_DATA" AS "ADS"), 
"NaiveBayes_data" AS 
(SELECT "Values"."KEY" AS "KEY", "Va

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
dataset = iris[,-5]

df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [9]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,Score_setosa,Score_versicolor,Score_virginica,Proba_setosa,Proba_versicolor,Proba_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
116,NA,NA,NA,3.720074e-44,6.128385e-07,9.999994e-01,-100.0000,-1.430516e+01,-6.128387e-07,virginica,0.9999994
87,NA,NA,NA,2.965568e-44,7.971795e-01,2.028205e-01,-100.2267,-2.266754e-01,-1.595434e+00,versicolor,0.7971795
71,NA,NA,NA,3.121382e-44,1.609361e-01,8.390639e-01,-100.1755,-1.826748e+00,-1.754684e-01,virginica,0.8390639
68,NA,NA,NA,3.720001e-44,9.999799e-01,2.007314e-05,-100.0000,-2.007334e-05,-1.081613e+01,versicolor,0.9999799
51,NA,NA,NA,2.983000e-44,8.018653e-01,1.981347e-01,-100.2208,-2.208147e-01,-1.618808e+00,versicolor,0.8018653
146,NA,NA,NA,3.720073e-44,7.915543e-07,9.999992e-01,-100.0000,-1.404927e+01,-7.915546e-07,virginica,0.9999992


# R Caret Rpart Output

In [11]:
pred_proba  =  predict(model, as.matrix(iris[,-5]), type = "prob")
df_r_out = data.frame(pred_proba)
names(df_r_out) = sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(dataset))
df_r_out$Score_setosa  =  NA
df_r_out$Score_versicolor  =  NA
df_r_out$Score_virginica  =  NA
df_r_out$LogProba_setosa  =  log(df_r_out$Proba_setosa)
df_r_out$LogProba_versicolor =  log(df_r_out$Proba_versicolor)
df_r_out$LogProba_virginica  =  log(df_r_out$Proba_virginica)
df_r_out$Decision =   predict(model, as.matrix(iris[,-5]), type = "raw")
df_r_out$DecisionProba =  apply(pred_proba, 1, function(x) max(x))
head(df_r_out)



Proba_setosa,Proba_versicolor,Proba_virginica,KEY,Score_setosa,Score_versicolor,Score_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
1,2.981309e-18,2.152373e-25,1,NA,NA,NA,0.000000e+00,-40.35417,-56.79806,setosa,1
1,3.169312e-17,6.938030e-25,2,NA,NA,NA,0.000000e+00,-37.99043,-55.62761,setosa,1
1,2.367113e-18,7.240956e-26,3,NA,NA,NA,0.000000e+00,-40.58486,-57.88746,setosa,1
1,3.069606e-17,8.690636e-25,4,NA,NA,NA,0.000000e+00,-38.02240,-55.40238,setosa,1
1,1.017337e-18,8.885794e-26,5,NA,NA,NA,0.000000e+00,-41.42934,-57.68276,setosa,1
1,2.717732e-14,4.344285e-21,6,NA,NA,NA,-2.720046e-14,-31.23639,-46.88543,setosa,1


# Compare R and SQL output

In [12]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,Score_setosa_1,Score_versicolor_1,Score_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2
1,1,2.981309e-18,2.152373e-25,NA,NA,NA,0.000000e+00,-40.35417,-56.79806,⋯,NA,NA,1,2.981309e-18,2.152373e-25,0.000000e+00,-40.35417,-56.79806,setosa,1
2,1,3.169312e-17,6.938030e-25,NA,NA,NA,0.000000e+00,-37.99043,-55.62761,⋯,NA,NA,1,3.169312e-17,6.938030e-25,0.000000e+00,-37.99043,-55.62761,setosa,1
3,1,2.367113e-18,7.240956e-26,NA,NA,NA,0.000000e+00,-40.58486,-57.88746,⋯,NA,NA,1,2.367113e-18,7.240956e-26,0.000000e+00,-40.58486,-57.88746,setosa,1
4,1,3.069606e-17,8.690636e-25,NA,NA,NA,0.000000e+00,-38.02240,-55.40238,⋯,NA,NA,1,3.069606e-17,8.690636e-25,0.000000e+00,-38.02240,-55.40238,setosa,1
5,1,1.017337e-18,8.885794e-26,NA,NA,NA,0.000000e+00,-41.42934,-57.68276,⋯,NA,NA,1,1.017337e-18,8.885794e-26,0.000000e+00,-41.42934,-57.68276,setosa,1
6,1,2.717732e-14,4.344285e-21,NA,NA,NA,-2.720046e-14,-31.23639,-46.88543,⋯,NA,NA,1,2.717732e-14,4.344285e-21,-2.708944e-14,-31.23639,-46.88543,setosa,1


In [13]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

KEY,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,Score_setosa_1,Score_versicolor_1,Score_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2


In [14]:
stopifnot(nrow(diffs_df) == 0)

In [15]:
summary(df_sql_out)

      KEY         Score_setosa   Score_versicolor Score_virginica
 Min.   :  1.00   Mode:logical   Mode:logical     Mode:logical   
 1st Qu.: 38.25   NA's:150       NA's:150         NA's:150       
 Median : 75.50                                                  
 Mean   : 75.50                                                  
 3rd Qu.:112.75                                                  
 Max.   :150.00                                                  
  Proba_setosa    Proba_versicolor    Proba_virginica    LogProba_setosa  
 Min.   :0.0000   Min.   :0.0000000   Min.   :0.000000   Min.   :-100.67  
 1st Qu.:0.0000   1st Qu.:0.0000000   1st Qu.:0.000000   1st Qu.:-100.00  
 Median :0.0000   Median :0.0000065   Median :0.001069   Median :-100.00  
 Mean   :0.3333   Mean   :0.3334777   Mean   :0.333189   Mean   : -66.09  
 3rd Qu.:1.0000   3rd Qu.:0.9770662   3rd Qu.:0.981799   3rd Qu.:   0.00  
 Max.   :1.0000   Max.   :0.9999997   Max.   :1.000000   Max.   :   0.00  
 LogProba_ver

In [16]:
summary(df_r_out)

  Proba_setosa    Proba_versicolor    Proba_virginica         KEY        
 Min.   :0.0000   Min.   :0.0000000   Min.   :0.000000   Min.   :  1.00  
 1st Qu.:0.0000   1st Qu.:0.0000000   1st Qu.:0.000000   1st Qu.: 38.25  
 Median :0.0000   Median :0.0000065   Median :0.001069   Median : 75.50  
 Mean   :0.3333   Mean   :0.3334777   Mean   :0.333189   Mean   : 75.50  
 3rd Qu.:1.0000   3rd Qu.:0.9770662   3rd Qu.:0.981799   3rd Qu.:112.75  
 Max.   :1.0000   Max.   :0.9999997   Max.   :1.000000   Max.   :150.00  
 Score_setosa   Score_versicolor Score_virginica LogProba_setosa 
 Mode:logical   Mode:logical     Mode:logical    Min.   :-703.6  
 NA's:150       NA's:150         NA's:150        1st Qu.:-344.9  
                                                 Median :-193.2  
                                                 Mean   :-209.0  
                                                 3rd Qu.:   0.0  
                                                 Max.   :   0.0  
 LogProba_versicolor